# 신용카드사기탐지_모델만들기

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import koreanize_matplotlib
import seaborn as sns
from sklearnex import patch_sklearn
patch_sklearn()

Extension for Scikit-learn* enabled (https://github.com/uxlfoundation/scikit-learn-intelex)


In [3]:
data = pd.read_csv("./data/fraud.csv")

In [4]:
data

,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,Moravian Falls,...,36.0788,-81.1781,3495,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0
1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,Orient,...,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0
2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,Malad City,...,42.1808,-112.2620,4154,Nature conservation officer,1962-01-19,a1a22d70485983eac12b5b88dad1cf95,1325376051,43.150704,-112.154481,0
3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,Boulder,...,46.2306,-112.1138,1939,Patent attorney,1967-01-12,6b849c168bdad6f867558c3793159a81,1325376076,47.034331,-112.561071,0
4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,Doe Hill,...,38.4207,-79.4629,99,Dance movement psychotherapist,1986-03-28,a41d7549acf90789359a9aa5346dcb46,1325376186,38.674999,-78.632459,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1852389,2020-12-31 23:59:07,30560609640617,fraud_Reilly and Sons,health_fitness,43.77,Michael,Olson,M,558 Michael Estates,Luray,...,40.4931,-91.8912,519,Town planner,1966-02-13,9b1f753c79894c9f4b71f04581835ada,1388534347,39.946837,-91.333331,0
1852390,2020-12-31 23:59:09,3556613125071656,fraud_Hoppe-Parisian,kids_pets,111.84,Jose,Vasquez,M,572 Davis Mountains,Lake Jackson,...,29.0393,-95.4401,28739,Futures trader,1999-12-27,2090647dac2c89a1d86c514c427f5b91,1388534349,29.661049,-96.186633,0
1852391,2020-12-31 23:59:15,6011724471098086,fraud_Rau-Robel,kids_pets,86.88,Ann,Lawson,F,144 Evans Islands Apt. 683,Burbank,...,46.1966,-118.9017,3684,Musician,1981-11-29,6c5b7c8add471975aa0fec023b2e8408,1388534355,46.658340,-119.715054,0
1852392,2020-12-31 23:59:24,4079773899158,fraud_Breitenberg LLC,travel,7.99,Eric,Preston,M,7020 Doyle Stream Apt. 951,Mesa,...,44.6255,-116.4493,129,Cartographer,1965-12-15,14392d723bb7737606b2700ac791b7aa,1388534364,44.470525,-117.080888,0


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1852394 entries, 0 to 1852393
Data columns (total 22 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   trans_date_trans_time  object 
 1   cc_num                 int64  
 2   merchant               object 
 3   category               object 
 4   amt                    float64
 5   first                  object 
 6   last                   object 
 7   gender                 object 
 8   street                 object 
 9   city                   object 
 10  state                  object 
 11  zip                    int64  
 12  lat                    float64
 13  long                   float64
 14  city_pop               int64  
 15  job                    object 
 16  dob                    object 
 17  trans_num              object 
 18  unix_time              int64  
 19  merch_lat              float64
 20  merch_long             float64
 21  is_fraud               int64  
dtypes: float64(5), int

In [6]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.3f}'.format)

In [7]:
data.head(2)

,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,state,zip,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.970,Jennifer,Banks,F,561 Perry Cove,Moravian Falls,NC,28654,36.079,-81.178,3495,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1325376018,36.011,-82.048,0
1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.230,Stephanie,Gill,F,43039 Riley Greens Suite 393,Orient,WA,99160,48.888,-118.210,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159,-118.186,0


In [8]:
data.head(100)

,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,state,zip,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.970,Jennifer,Banks,F,561 Perry Cove,Moravian Falls,NC,28654,36.079,-81.178,3495,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1325376018,36.011,-82.048,0
1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.230,Stephanie,Gill,F,43039 Riley Greens Suite 393,Orient,WA,99160,48.888,-118.210,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159,-118.186,0
2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.110,Edward,Sanchez,M,594 White Dale Suite 530,Malad City,ID,83252,42.181,-112.262,4154,Nature conservation officer,1962-01-19,a1a22d70485983eac12b5b88dad1cf95,1325376051,43.151,-112.154,0
3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.000,Jeremy,White,M,9443 Cynthia Court Apt. 038,Boulder,MT,59632,46.231,-112.114,1939,Patent attorney,1967-01-12,6b849c168bdad6f867558c3793159a81,1325376076,47.034,-112.561,0
4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.960,Tyler,Garcia,M,408 Bradley Rest,Doe Hill,VA,24433,38.421,-79.463,99,Dance movement psychotherapist,1986-03-28,a41d7549acf90789359a9aa5346dcb46,1325376186,38.675,-78.632,0
5,2019-01-01 00:04:08,4767265376804500,"fraud_Stroman, Hudson and Erdman",gas_transport,94.630,Jennifer,Conner,F,4655 David Island,Dublin,PA,18917,40.375,-75.204,2158,Transport planner,1961-06-19,189a841a0a8ba03058526bcfe566aab5,1325376248,40.653,-76.153,0
6,2019-01-01 00:04:42,30074693890476,fraud_Rowe-Vandervort,grocery_net,44.540,Kelsey,Richards,F,889 Sarah Station Suite 624,Holcomb,KS,67851,37.993,-100.989,2691,Arboriculturist,1993-08-16,83ec1cc84142af6e2acf10c44949e720,1325376282,37.163,-100.153,0
7,2019-01-01 00:05:08,6011360759745864,fraud_Corwin-Collins,gas_transport,71.650,Steven,Williams,M,231 Flores Pass Suite 720,Edinburg,VA,22824,38.843,-78.600,6018,"Designer, multimedia",1947-08-21,6d294ed2cc447d2c71c7171a3d54967c,1325376308,38.948,-78.540,0
8,2019-01-01 00:05:18,4922710831011201,fraud_Herzog Ltd,misc_pos,4.270,Heather,Chase,F,6888 Hicks Stream Suite 954,Manor,PA,15665,40.336,-79.661,1472,Public affairs consultant,1941-03-07,fc28024ce480f8ef21a32d64c93a29f5,1325376318,40.352,-79.958,0
9,2019-01-01 00:06:01,2720830304681674,"fraud_Schoen, Kuphal and Nitzsche",grocery_pos,198.390,Melissa,Aguilar,F,21326 Taylor Squares Suite 708,Clarksville,TN,37040,36.522,-87.349,151785,Pathologist,1974-03-28,3b9014ea8fb80bd65de0b1463b00b00e,1325376361,37.179,-87.485,0


In [9]:
data.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1852394 entries, 0 to 1852393
Data columns (total 22 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   trans_date_trans_time  1852394 non-null  object 
 1   cc_num                 1852394 non-null  int64  
 2   merchant               1852394 non-null  object 
 3   category               1852394 non-null  object 
 4   amt                    1852394 non-null  float64
 5   first                  1852394 non-null  object 
 6   last                   1852394 non-null  object 
 7   gender                 1852394 non-null  object 
 8   street                 1852394 non-null  object 
 9   city                   1852394 non-null  object 
 10  state                  1852394 non-null  object 
 11  zip                    1852394 non-null  int64  
 12  lat                    1852394 non-null  float64
 13  long                   1852394 non-null  float64
 14  city_pop          

In [10]:
data[['amt', 'city_pop']].describe()

,amt,city_pop
count,1852394.000,1852394.000
mean,70.064,88643.675
std,159.254,301487.618
min,1.000,23.000
25%,9.640,741.000
50%,47.450,2443.000
75%,83.100,20328.000
max,28948.900,2906700.000


In [11]:
data.isna().sum()

trans_date_trans_time    0
cc_num                   0
merchant                 0
category                 0
amt                      0
first                    0
last                     0
gender                   0
street                   0
city                     0
state                    0
zip                      0
lat                      0
long                     0
city_pop                 0
job                      0
dob                      0
trans_num                0
unix_time                0
merch_lat                0
merch_long               0
is_fraud                 0
dtype: int64

In [12]:
obj_cols = data.select_dtypes(include='object')
num_cols = data.select_dtypes(exclude='object')

In [13]:
obj_cols.columns

Index(['trans_date_trans_time', 'merchant', 'category', 'first', 'last',
       'gender', 'street', 'city', 'state', 'job', 'dob', 'trans_num'],
      dtype='object')

In [14]:
for col in obj_cols:
    print("=" * 30, col, "=" * 30)
    print("nunique:" ,obj_cols[col].nunique(), end="\n\n")
    print(obj_cols[col].value_counts(), end="\n\n")
    print(data[[col, 'is_fraud']].groupby(col).mean().sort_values(by='is_fraud', ascending=False), end="\n\n")

============================== trans_date_trans_time ==============================
nunique: 1819551

trans_date_trans_time
2019-04-22 16:02:01    4
2020-06-01 01:37:47    4
2020-06-02 12:47:07    4
2020-10-05 19:37:49    4
2020-12-13 17:53:47    4
                      ..
2019-01-01 00:41:45    1
2019-01-01 00:42:26    1
2019-01-01 00:43:38    1
2019-01-01 00:44:20    1
2020-12-31 23:50:17    1
Name: count, Length: 1819551, dtype: int64

                       is_fraud
trans_date_trans_time          
2019-09-11 22:58:11       1.000
2019-12-22 01:09:30       1.000
2019-02-12 23:49:17       1.000
2019-09-23 06:33:47       1.000
2019-05-03 04:38:52       1.000
...                         ...
2019-01-01 00:41:45       0.000
2019-01-01 00:42:26       0.000
2019-01-01 00:43:38       0.000
2019-01-01 00:44:20       0.000
2020-12-31 23:50:17       0.000

[1819551 rows x 1 columns]

============================== merchant ==============================
nunique: 693

merchant
fraud_Kilback LLC 

                          is_fraud
job                               
Accountant, chartered        1.000
Warehouse manager            1.000
Careers adviser              1.000
Solicitor                    1.000
Ship broker                  1.000
...                            ...
Health physicist             0.001
Contractor                   0.000
Engineer, communications     0.000
Engineer, drilling           0.000
Stage manager                0.000

[497 rows x 1 columns]

============================== dob ==============================
nunique: 984

dob
1977-03-23    8044
1988-09-15    6574
1981-08-29    6571
1955-05-06    5121
1960-01-13    4395
              ... 
1962-03-14       7
1944-06-17       7
1970-06-25       7
1932-05-09       7
1998-08-02       6
Name: count, Length: 984, dtype: int64

            is_fraud
dob                 
2001-07-17     1.000
2002-03-17     1.000
1925-08-29     1.000
1927-02-03     1.000
1944-05-30     1.000
...              ...
1949-10-04     0.00

In [ ]:
data[data['state'] == 'DE']

'trans_date_trans_time', 'merchant', 'category', 'gender', 'job' 

In [15]:
from ydata_profiling import ProfileReport

C:\Users\vbak0\miniforge3\envs\fintech\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
profile = ProfileReport(data, title="Profiling Report")

In [17]:
profile.to_file("is_froad_eda.html")

Export report to file: 100%|█████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 21.87it/s]


* 집 위치와 상점 위치가 서로 연관
* cc_num의 유일값이 999개

In [18]:
data['cc_num'].nunique()

999

is_fraud의 비율

In [19]:
data['is_fraud'].mean()

np.float64(0.005210014716091717)

# 카드별 (고객) 거래 금액에서의 이상치 찾기, is_fraud와 관계 확인

In [20]:
amt_desc = data[['cc_num', 'amt']].groupby('cc_num').describe()

In [21]:
amt_desc

amt                                               \
                       count    mean      std   min    25%    50%     75%   
cc_num                                                                      
60416207185         2196.000  59.258  142.870 1.020  7.560 36.540  72.802   
60422928733         2200.000  65.483   92.043 1.010  9.075 51.405  87.085   
60423098130          738.000  96.376 1000.694 1.010  8.780 35.425  75.463   
60427851591          743.000 107.488  131.015 8.860 59.140 76.670  99.110   
60487002085          735.000  64.097  153.208 1.020  8.095 35.670  70.855   
...                      ...     ...      ...   ...    ...    ...     ...   
4958589671582726883 2191.000  67.206  137.504 1.000  8.830 50.130  84.455   
4973530368125489546 1467.000  75.789  258.847 1.010  9.435 52.660  88.340   
4980323467523543940  736.000  70.709  119.903 1.010  8.465 42.515  83.458   
4989847570577635369 1471.000  93.009  128.397 1.030 19.940 57.170 111.945   
4992346398065154184 2922.000  66.456  140.983 1.020  9.062 41.960  82.990   

                               
                          max  
cc_num                         
60416207185          3437.460  
60422928733          1290.370  
60423098130         27119.770  
60427851591          1164.360  
60487002085          2758.460  
...                       ...  
4958589671582726883  4292.860  
4973530368125489546  8749.440  
4980323467523543940  1327.430  
4989847570577635369  1223.850  
4992346398065154184  4669.070  

[999 rows x 8 columns]

In [22]:
amt_desc = data.groupby(['cc_num'])['amt'].describe()

In [23]:
amt_desc

,count,mean,std,min,25%,50%,75%,max
cc_num,,,,,,,,
60416207185,2196.000,59.258,142.870,1.020,7.560,36.540,72.802,3437.460
60422928733,2200.000,65.483,92.043,1.010,9.075,51.405,87.085,1290.370
60423098130,738.000,96.376,1000.694,1.010,8.780,35.425,75.463,27119.770
60427851591,743.000,107.488,131.015,8.860,59.140,76.670,99.110,1164.360
60487002085,735.000,64.097,153.208,1.020,8.095,35.670,70.855,2758.460
...,...,...,...,...,...,...,...,...
4958589671582726883,2191.000,67.206,137.504,1.000,8.830,50.130,84.455,4292.860
4973530368125489546,1467.000,75.789,258.847,1.010,9.435,52.660,88.340,8749.440
4980323467523543940,736.000,70.709,119.903,1.010,8.465,42.515,83.458,1327.430


In [24]:
amt_desc['iqr_max'] = amt_desc['75%'] + (1.5 * (amt_desc['75%'] - amt_desc['25%']))

In [25]:
amt_desc = amt_desc.reset_index()
amt_desc

,cc_num,count,mean,std,min,25%,50%,75%,max,iqr_max
0,60416207185,2196.000,59.258,142.870,1.020,7.560,36.540,72.802,3437.460,170.666
1,60422928733,2200.000,65.483,92.043,1.010,9.075,51.405,87.085,1290.370,204.100
2,60423098130,738.000,96.376,1000.694,1.010,8.780,35.425,75.463,27119.770,175.486
3,60427851591,743.000,107.488,131.015,8.860,59.140,76.670,99.110,1164.360,159.065
4,60487002085,735.000,64.097,153.208,1.020,8.095,35.670,70.855,2758.460,164.995
...,...,...,...,...,...,...,...,...,...,...
994,4958589671582726883,2191.000,67.206,137.504,1.000,8.830,50.130,84.455,4292.860,197.892
995,4973530368125489546,1467.000,75.789,258.847,1.010,9.435,52.660,88.340,8749.440,206.697
996,4980323467523543940,736.000,70.709,119.903,1.010,8.465,42.515,83.458,1327.430,195.946
997,4989847570577635369,1471.000,93.009,128.397,1.030,19.940,57.170,111.945,1223.850,249.952


# 원본 데이터와 그룹 연산 결과 합치기

In [26]:
amt_desc.columns

Index(['cc_num', 'count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max',
       'iqr_max'],
      dtype='object')

In [27]:
amt_desc.columns = ['cc_num', 'count', 'amt_mean', 'amt_std', 'min', '25%', '50%', '75%', 'max',
       'amt_iqr_max']

In [28]:
data = pd.merge(data, amt_desc[['cc_num', 'amt_std', 'amt_mean', 'amt_iqr_max']], how='left', on='cc_num')
data

,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,state,zip,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud,amt_std,amt_mean,amt_iqr_max
0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.970,Jennifer,Banks,F,561 Perry Cove,Moravian Falls,NC,28654,36.079,-81.178,3495,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1325376018,36.011,-82.048,0,127.530,89.409,256.620
1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.230,Stephanie,Gill,F,43039 Riley Greens Suite 393,Orient,WA,99160,48.888,-118.210,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159,-118.186,0,159.202,56.078,158.060
2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.110,Edward,Sanchez,M,594 White Dale Suite 530,Malad City,ID,83252,42.181,-112.262,4154,Nature conservation officer,1962-01-19,a1a22d70485983eac12b5b88dad1cf95,1325376051,43.151,-112.154,0,116.689,69.924,210.800
3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.000,Jeremy,White,M,9443 Cynthia Court Apt. 038,Boulder,MT,59632,46.231,-112.114,1939,Patent attorney,1967-01-12,6b849c168bdad6f867558c3793159a81,1325376076,47.034,-112.561,0,280.078,80.090,193.098
4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.960,Tyler,Garcia,M,408 Bradley Rest,Doe Hill,VA,24433,38.421,-79.463,99,Dance movement psychotherapist,1986-03-28,a41d7549acf90789359a9aa5346dcb46,1325376186,38.675,-78.632,0,94.323,95.341,149.974
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1852389,2020-12-31 23:59:07,30560609640617,fraud_Reilly and Sons,health_fitness,43.770,Michael,Olson,M,558 Michael Estates,Luray,MO,63453,40.493,-91.891,519,Town planner,1966-02-13,9b1f753c79894c9f4b71f04581835ada,1388534347,39.947,-91.333,0,110.821,62.348,182.535
1852390,2020-12-31 23:59:09,3556613125071656,fraud_Hoppe-Parisian,kids_pets,111.840,Jose,Vasquez,M,572 Davis Mountains,Lake Jackson,TX,77566,29.039,-95.440,28739,Futures trader,1999-12-27,2090647dac2c89a1d86c514c427f5b91,1388534349,29.661,-96.187,0,168.361,50.452,136.045
1852391,2020-12-31 23:59:15,6011724471098086,fraud_Rau-Robel,kids_pets,86.880,Ann,Lawson,F,144 Evans Islands Apt. 683,Burbank,WA,99323,46.197,-118.902,3684,Musician,1981-11-29,6c5b7c8add471975aa0fec023b2e8408,1388534355,46.658,-119.715,0,119.949,88.704,261.548
1852392,2020-12-31 23:59:24,4079773899158,fraud_Breitenberg LLC,travel,7.990,Eric,Preston,M,7020 Doyle Stream Apt. 951,Mesa,ID,83643,44.626,-116.449,129,Cartographer,1965-12-15,14392d723bb7737606b2700ac791b7aa,1388534364,44.471,-117.081,0,89.526,60.998,186.940


In [29]:
data.columns

Index(['trans_date_trans_time', 'cc_num', 'merchant', 'category', 'amt',
       'first', 'last', 'gender', 'street', 'city', 'state', 'zip', 'lat',
       'long', 'city_pop', 'job', 'dob', 'trans_num', 'unix_time', 'merch_lat',
       'merch_long', 'is_fraud', 'amt_std', 'amt_mean', 'amt_iqr_max'],
      dtype='object')

In [30]:
data = data.drop(['first', 'last', 'street', 'city', 'state', 'zip', 'trans_num', 'unix_time'], axis=1)

In [31]:
data

,trans_date_trans_time,cc_num,merchant,category,amt,gender,lat,long,city_pop,job,dob,merch_lat,merch_long,is_fraud,amt_std,amt_mean,amt_iqr_max
0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.970,F,36.079,-81.178,3495,"Psychologist, counselling",1988-03-09,36.011,-82.048,0,127.530,89.409,256.620
1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.230,F,48.888,-118.210,149,Special educational needs teacher,1978-06-21,49.159,-118.186,0,159.202,56.078,158.060
2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.110,M,42.181,-112.262,4154,Nature conservation officer,1962-01-19,43.151,-112.154,0,116.689,69.924,210.800
3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.000,M,46.231,-112.114,1939,Patent attorney,1967-01-12,47.034,-112.561,0,280.078,80.090,193.098
4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.960,M,38.421,-79.463,99,Dance movement psychotherapist,1986-03-28,38.675,-78.632,0,94.323,95.341,149.974
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1852389,2020-12-31 23:59:07,30560609640617,fraud_Reilly and Sons,health_fitness,43.770,M,40.493,-91.891,519,Town planner,1966-02-13,39.947,-91.333,0,110.821,62.348,182.535
1852390,2020-12-31 23:59:09,3556613125071656,fraud_Hoppe-Parisian,kids_pets,111.840,M,29.039,-95.440,28739,Futures trader,1999-12-27,29.661,-96.187,0,168.361,50.452,136.045
1852391,2020-12-31 23:59:15,6011724471098086,fraud_Rau-Robel,kids_pets,86.880,F,46.197,-118.902,3684,Musician,1981-11-29,46.658,-119.715,0,119.949,88.704,261.548
1852392,2020-12-31 23:59:24,4079773899158,fraud_Breitenberg LLC,travel,7.990,M,44.626,-116.449,129,Cartographer,1965-12-15,44.471,-117.081,0,89.526,60.998,186.940


In [32]:
# 이상값 찾기
data.loc[data['amt'] > data['amt_iqr_max'], 'is_fraud'].mean() * 100

InvalidIndexError: (0          False
1          False
2           True
3          False
4          False
           ...  
1852389    False
1852390    False
1852391    False
1852392    False
1852393    False
Length: 1852394, dtype: bool, 'is_fraud')

# 사용 금액의 z-score를 구한 후 사기 비율 계산
* zscore: (개별 값 - 평균) / 표준편차 = -3 > , 3 > => 이상값


In [ ]:
data['amt_zscore'] = (data['amt'] - data['amt_mean']) / data['amt_std']

In [ ]:
data

In [ ]:
data[(data['amt_zscore'] < -3) | (data['amt_zscore'] > 3)]['is_fraud'].mean() * 100

In [ ]:
data[(data['amt_zscore'] < -3) | (data['amt_zscore'] > 3)]['is_fraud'].value_counts()

# 사용자가의 주된 지출처
* 식품에 지출이 많은 사람, 레저용품에 지출을 갑자기 많이 한다면 의심스러움
* 평소 지출하는 카테고리 혹은 가게가 아닌 경우
* 카드 번호별, 카테고리별 지출 평균, 표준편차

In [ ]:
data.columns

In [ ]:
cat_amt = data.groupby(['cc_num', 'category'])['amt'].agg(['mean', 'std']).reset_index()
cat_amt

In [ ]:
cat_amt.columns = ['cc_num', 'category', 'cat_amt_mean', 'cat_amt_std']

In [ ]:
cat_amt

In [ ]:
data = pd.merge(data, cat_amt, how='left', on=['cc_num', 'category'])
data

In [ ]:
#z-score 계산

data['cat_amt_zscore'] = (data['amt'] - data['cat_amt_mean']) / data['cat_amt_std']

In [ ]:
data['cat_amt_zscore']

In [ ]:
data[(data['cat_amt_zscore'] < -3) | (data['cat_amt_zscore'] > 3)]['is_fraud'].mean() * 100

# 집의 위치와 가게의 위치 상관관계 높음
* 집과 가게의 거리를 위도/경도 데이터를 이용하여 계산, 평균 거리를 초과하는 곳에서 발생한 거래를 통해 부당 거래 검출

In [ ]:
data.columns

# geopy를 이용해 위도/경도 정보로 집과 가게의 거리 계산하기

In [ ]:
from geopy.distance import distance

In [ ]:
data.columns

In [ ]:
data['is_fraud'].apply()

In [ ]:
data.loc[:3, :].apply(lambda x: print(x))

In [ ]:
data['home_merch_dist'] = data.apply(lambda x: distance((x['lat'], x['long']), (x['merch_lat'], x['merch_long'])).km, axis=1)
data['home_merch_dist']

In [ ]:
print(distance(((data['lat'][0]), data['long'][0]), (data['merch_lat'][0], data['merch_long'][0])).km)

In [ ]:
dist_desc = data.groupby(['cc_num'])['home_merch_dist'].agg(['mean', 'std']).reset_mean()
dist_desc

In [ ]:
dist_desc.columns = ['cc_num', 'dist_mean', 'dist_std']

In [ ]:
data = pd.merge(data, dist_desc, how='left', on='cc_num')
data

In [ ]:
data['dist_zscore'] = (data['home_merch_dist']- data['dist_mean']) / data['dist_std']

In [ ]:
data[(data['dist_zscore'] < -3) | (data['dist_zscore'] > 3)]

In [ ]:
data.columns

In [ ]:
data.drop(['cc_num','merchant','amt_mean', 'amt_std', 'cat_amt_mean', 'cat_amt_std',
          'dist_mean', 'dist_std', 'lat', 'long', 'merch_lat', 'merch_long'], axis=1)

# 나이 추출

In [ ]:
data['trans_date_trans_time'][-1:][1852393][:4]

In [ ]:
data['age'] = data['dob'].apply(lambda x: 2020 - int(x[:4]))

In [ ]:
data.groupby('age')['is_fraud'].mean().sort_values(ascending=False)

In [ ]:
data['age'].value_counts()

In [ ]:
data.drop(['trans_date_trans_time', 'dob'], axis=1)

# 종속변수, 독립변수 분리, 훈련/테스트 데이터로 나누기

In [47]:
X = data.drop('is_fraud', axis=1)
y = data['is_fraud']

In [48]:
X.columns

Index(['trans_date_trans_time', 'cc_num', 'merchant', 'category', 'amt',
       'gender', 'lat', 'long', 'city_pop', 'job', 'dob', 'merch_lat',
       'merch_long', 'amt_std', 'amt_mean', 'amt_iqr_max'],
      dtype='object')

In [49]:
X = pd.get_dummies(X, drop_first=True)

MemoryError: Unable to allocate 3.07 TiB for an array with shape (1852394, 1819551) and data type bool

In [39]:
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report, roc_auc_score


* 홀드아웃, train, valid, test

In [40]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.4, stratify=y, random_state=42)
X_valid, X_test, y_valid, y_test = train_test_split(X_valid, y_valid, test_size=0.5, stratify=y_valid, random_state=42)

In [41]:
print(len(X_train), len(X_valid), len(X_test))

1111436 370479 370479


In [42]:
print(y_train.value_counts())

is_fraud
0    1105645
1       5791
Name: count, dtype: int64


In [43]:
print(y_test.value_counts())

is_fraud
0    368549
1      1930
Name: count, dtype: int64


In [44]:
print(y_valid.value_counts())

is_fraud
0    368549
1      1930
Name: count, dtype: int64


의사결정 나무로 분석하기

In [45]:
dtc = DecisionTreeClassifier(class_weight='balanced', random_state=42)
dtc.fit(X_train, y_train)
pred = dtc.predict(X_valid)
print(classification_report(y_valid, pred))
print("=" * 30, test_result, "=" * 30)
test_pred = dtc.predict(X_test)
print(classification_report(y_test, test_pred))


ValueError: could not convert string to float: '2019-02-14 19:17:36'

In [ ]:
print(dtc.get_params())

In [ ]:
importance = pd.DataFrame(dtc.feature_importances_, columns=['importance'])
colum_name = pd.DateFrame(dtc.feature_names_in_, columns=['column'])

In [ ]:
dtc_im_col = pd.concat([colum_name, importance], axis=1)
dtc_im_col

In [ ]:
for i in range(1, 11):
    dtc = DecisionTreeClassifier(class_weight='balanced', random_state=42)
    dtc.fit(X_train, y_train)
    pred = dtc.predict(X_valid)
    print(classification_report(y_valid, pred))
    print("=" * 30, f"test_result {i}", "=" * 30)
    test_pred = dtc.predict(X_test)
    print(classification_report(y_test, test_pred))
    print()

random_forest로 분석하기

In [50]:
rfc = RandomForestClassifier(class_weight="balanced", random_state=42, n_jobs=10)
rfc.fit(X_train, y_train)
pred = rfc.predict(X_valid)
print("=" * 30, "valid_result", "=" * 30)
print(classification_report(y_valid, pred), end="\n\n")

print("=" * 30, "test_result", "=" * 30)
test_pred = rfc.predict(X_test)
print(classification_report(y_test, test_pred), end="\n\n")

ValueError: could not convert string to float: '2019-02-14 19:17:36'

In [ ]:
# 각 트리의 실제 max_depth 출력
depths = [estimator.tree_.max_depth for estimator in rfc.estimators_]
print("각 트리의 최대 깊이 목록:", depths)
print("평균 깊이:", sum(depths) / len(depths))
print("최댓값:", max(depths), " / 최솟값:", min(depths))

XGBOOST로 분석하기

In [55]:
y_train.value_counts()

is_fraud
0    1105645
1       5791
Name: count, dtype: int64

* XGBoost에서 소수 클래스에 가중치줄 때 값 계산
* n_negative / n_positive
* scale_pos_weight = n_negative / n_positive

In [54]:
1105645 / 5791

190.92471075807288

In [56]:
xgb = XGBClassifier(scale_pos_weight=190, n_jobs=10, random_state=42)

xgb.fit(X_train, y_train)
pred = xgb.predict(X_valid)
print("=" * 30, "valid_result", "=" * 30)
print(classification_report(y_valid, pred), end="\n\n")

print("=" * 30, "test_result", "=" * 30)
test_pred = xgb.predict(X_test)
print(classification_report(y_test, test_pred), end="\n\n")

ValueError: DataFrame.dtypes for data must be int, float, bool or category. When categorical type is supplied, the experimental DMatrix parameter`enable_categorical` must be set to `True`.  Invalid columns:trans_date_trans_time: object, merchant: object, category: object, gender: object, job: object, dob: object

catboost

In [ ]:
cbc = CatBoostClassifier(class_weight=[1, 190], n_jobs=10, random_state=42)

cbc.fit(X_train, y_train)
pred = cbc.predict(X_valid)
print("=" * 30, "valid_result", "=" * 30)
print(classification_report(y_valid, pred), end="\n\n")

print("=" * 30, "test_result", "=" * 30)
test_pred = cbc.predict(X_test)
print(classification_report(y_test, test_pred), end="\n\n")

In [ ]:
new_cols = []
for idx, col in enumerate(X.columns):
    new_cols.append(col.replace(", ", "_").replace("/", "_").replace(" ", "_").replace("(", "").replace(")", "").replace("'", ""))


In [ ]:
print(len(new_cols))
print(len(X.columns))


In [ ]:
X.columns = new_cols
X.columns

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.4, stratify=y, random_state=42)
X_valid, X_test, y_valid, y_test = train_test_split(X_valid, y_valid, test_size=0.5, stratify=y_valid, random_state=42)

Lightgbm

In [57]:
lgbm = LGBMClassifier(class_weight="balanced", n_jobs=10, random_state=42)

lgbm.fit(X_train, y_train)
pred = lgbm.predict(X_valid)
print("=" * 30, "valid_result", "=" * 30)
print(classification_report(y_valid, pred), end="\n\n")

print("=" * 30, "test_result", "=" * 30)
test_pred = lgbm.predict(X_test)
print(classification_report(y_test, test_pred), end="\n\n")

ValueError: pandas dtypes must be int, float or bool.
Fields with bad pandas dtypes: trans_date_trans_time: object, merchant: object, category: object, gender: object, job: object, dob: object

# 랜덤포레스트로 하이퍼파라미터 튜닝하기

In [ ]:
rfc = RandomForestClssifier(class_weight="balanced", random_state=42, n_jobs=10)

In [ ]:
params = dict(criterion=["gini", "entropy", "log_loss"],
              max_depth=[150, 160, 170, 180, 200],
              random_state=[42],
              class_weight=[None, "balanced"])


In [ ]:
%%time
rand_cv = RandomizedSearchCV(rfc, param_distributions=params, scoring='f1', cv=5, n_jobs=10, random_state=42)
rand_cv.fit(X_train, y_train)
pred = rand_cv.predict(X_valid)
print("=" * 30, "valid_result", "=" * 30)
print("best_params: ", rand_cv.best_params_)
print("best_score_: ", rand_cv.best_score_)
print(classification_report(y_valid, pred), end="\n\n")
print("=" * 30, "test_result", "=" * 30)
test_pred = rand_cv.predict(X_test)
print(classification_report(y_test, test_pred), end="\n\n")


# 금융사기란? (financial fraud)
* 금융 거래에서 속이거나 기만해서 자산이나 돈을 획득하는 행위
* 주요 유형
    * 신용카드 사기
    * 피싱, 신분 도용
    * 보험 사기
    * 돈세탁

# 금융 사기 대응법
* 고객의 거래 패턴 분석, 비정상 거래 포착
* 전문가 기반 규칙 - 전문가들의 경험에 의한 사기 탐지
* 통계 기반 분석 - 수학적 모델, 통계학 기반 모델로 이상 탐지 수행
* 머신러닝 기반 분석 - 대규모 데이터 학습, 전문가 기반, 통계 기반으로 찾지 못하는 사기도 탐지 가능

# 금융 사기 탐지의 어려움
* 사기 확인이 피해자 불만 표현을 통해서 이루어지는 경우가 많음
* 사기 거래 자체가 적음 => 데이터 확보가 어려움
* 데이터 부족으로 인해 성능이 좋은 모델 생성이 어려움
* 가상 데이터를 인공적으로 생성해서 모델 생성에 활용

# 사기 방지와 고객 경험의 균형
* 정당한 거래가 차단되어 고객에게 좋지 않은 사용 경험을 주는 경우
* 엄격한 탐지 -> 사용자 불편 증가
* 관대한 탐지 -> 사기율 상승
* 적절한 타협이 필요